### 矢量存储和检索器

本教程将让您熟悉LangChain的向量存储和检索器抽象。

这些抽象旨在支持从（向量）数据库和其他来源检索数据，以便与 LLM 工作流集成。

它们对于获取数据作为模型推理的一部分进行推理的应用程序非常重要，

例如检索增强生成或 RAG（请参阅此处的 RAG 教程）。

#### 概念

本指南重点介绍文本数据的检索。我们将介绍以下概念：


文件;

向量存储;

猎犬。

#### 设置
Jupyter 笔记本

本教程和其他教程可能在 Jupyter 笔记本中运行最方便。有关如何安装的说明，请参阅此处。

##### 安装
本教程需要安装 `langchain`, `langchain-chroma` `langchain-openai`

#### 文件
LangChain实现了一个文档抽象，旨在表示文本单元和关联的元数据。它有两个属性：

`page_content`：表示内容的字符串;

`metadata`：包含任意元数据的字典。

该属性可以捕获有关文档源、文档与其他文档的关系以及其他信息的信息。

请注意，单个对象通常表示较大文档的块。metadataDocument

让我们生成一些示例文档

In [11]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="狗是伟大的伴侣，以其忠诚和友好而闻名.",
        metadata={"source": "哺乳动物宠物-doc"},
    ),
    Document(
        page_content="猫是独立的宠物，经常享受自己的空间.",
        metadata={"source": "哺乳动物宠物-doc"},
    ),
    Document(
        page_content="金鱼是初学者的热门宠物，需要相对简单的照顾.",
        metadata={"source": "鱼类动物-doc"},
    ),
    Document(
        page_content="鹦鹉是聪明的鸟，能够模仿人类的语言.",
        metadata={"source": "鸟类动物-doc"},
    ),
    Document(
        page_content="兔子是群居动物，需要足够的空间来跳来跳去.",
        metadata={"source": "哺乳动物宠物-doc"},
    ),
]

在这里，我们生成了五个文档，其中包含指示三个不同“来源”的元数据。

#### 矢量存储
矢量搜索是存储和搜索非结构化数据（如非结构化文本）的常用方法。这个想法是存储与文本关联的数字向量。

给定一个查询，我们可以将其嵌入为相同维度的向量，并使用向量相似度量来识别存储中的相关数据。

LangChain VectorStore 对象包含用于将文本和对象添加到存储中的方法，以及使用各种相似度指标查询它们的方法。

它们通常使用嵌入模型进行初始化，这些模型决定了如何将文本数据转换为数字向量。Document

LangChain包括一套与不同向量存储技术的集成。

一些矢量存储由提供商（例如，各种云提供商）托管，需要特定的凭据才能使用;

有些（如Postgres）运行在单独的基础设施中，可以在本地或通过第三方运行;其他人可以在内存中运行轻量级工作负载。

在这里，我们将演示使用Chroma使用LangChain VectorStores，其中包括内存实现。

为了实例化向量存储，我们通常需要提供一个嵌入模型来指定如何将文本转换为数字向量。在这里，我们将使用 OpenAI 嵌入。(实际上是用阿里的来)

In [3]:
import os,getpass
os.environ["DASHSCOPE_API_KEY"] = getpass.getpass()

In [12]:
from langchain_chroma import Chroma
from langchain_community.embeddings import DashScopeEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=DashScopeEmbeddings(),
)

在此处调用会将文档添加到矢量存储中。

`VectorStore` 实现用于添加文档的方法，这些文档也可以在实例化对象后调用。

大多数实现都允许您连接到现有的向量存储，例如，通过提供客户端、索引名称或其他信息。有关更多详细信息，请参阅特定集成的文档。`.from_documents`

一旦我们实例化了包含文档的`a`，我们就可以查询它。

VectorStore 包括用于查询的方法：`VectorStore`

同步和异步;

通过字符串查询和向量;

有和没有返回相似性分数;

通过相似性和最大边际相关性（以平衡相似性与查询到检索结果的多样性）。

这些方法通常会在其输出中包含 Document 对象的列表。

#### 例子

根据与字符串查询的相似性返回文档：

In [13]:
vectorstore.similarity_search("猫")

[Document(page_content='猫是独立的宠物，经常享受自己的空间.', metadata={'source': '哺乳动物宠物-doc'}),
 Document(page_content='狗是伟大的伴侣，以其忠诚和友好而闻名.', metadata={'source': '哺乳动物宠物-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='兔子是群居动物，需要足够的空间来跳来跳去.', metadata={'source': '哺乳动物宠物-doc'})]

In [14]:
# 异步查询：
await vectorstore.asimilarity_search("猫")

[Document(page_content='猫是独立的宠物，经常享受自己的空间.', metadata={'source': '哺乳动物宠物-doc'}),
 Document(page_content='狗是伟大的伴侣，以其忠诚和友好而闻名.', metadata={'source': '哺乳动物宠物-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='兔子是群居动物，需要足够的空间来跳来跳去.', metadata={'source': '哺乳动物宠物-doc'})]

In [15]:
# 返回相似度:

# 请注意，提供商实施的分数不同；此处为Chroma。
# 返回应与成反比的距离度量。
# 相似性。

vectorstore.similarity_search_with_score("猫")

[(Document(page_content='猫是独立的宠物，经常享受自己的空间.', metadata={'source': '哺乳动物宠物-doc'}),
  6114.28076171875),
 (Document(page_content='狗是伟大的伴侣，以其忠诚和友好而闻名.', metadata={'source': '哺乳动物宠物-doc'}),
  10852.2373046875),
 (Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  10947.896484375),
 (Document(page_content='兔子是群居动物，需要足够的空间来跳来跳去.', metadata={'source': '哺乳动物宠物-doc'}),
  12230.74609375)]

In [16]:
# 根据与嵌入查询的相似性返回文档

embedding = DashScopeEmbeddings().embed_query("猫")

vectorstore.similarity_search_by_vector(embedding)

[Document(page_content='猫是独立的宠物，经常享受自己的空间.', metadata={'source': '哺乳动物宠物-doc'}),
 Document(page_content='狗是伟大的伴侣，以其忠诚和友好而闻名.', metadata={'source': '哺乳动物宠物-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='兔子是群居动物，需要足够的空间来跳来跳去.', metadata={'source': '哺乳动物宠物-doc'})]

#### Retrievers 追回

LangChain 对象不对 `Runnable` 进行子类化，因此无法立即集成到 `LangChain` 表达式语言链中。`VectorStore`

LangChain检索器是可运行的，因此它们实现了一组标准方法（例如，同步和异步以及操作），并被设计为合并到LCEL链中。`invokebatch`

我们可以自己创建一个简单的版本，而无需子类化。

如果我们选择我们希望使用哪种方法来检索文档，我们可以轻松创建一个可运行的。

下面我们将围绕该方法构建一个：`Retriever`  `similarity_search`

In [17]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["猫", "鲨鱼"])

[[Document(page_content='猫是独立的宠物，经常享受自己的空间.', metadata={'source': '哺乳动物宠物-doc'})],
 [Document(page_content='金鱼是初学者的热门宠物，需要相对简单的照顾.', metadata={'source': '鱼类动物-doc'})]]

`Vectorstores` 实现一个将生成 Retriever（特别是 VectorStoreRetriever）的方法。

这些检索器包括特定的属性，用于标识要调用的基础向量存储的哪些方法，以及如何参数化它们。

例如，我们可以使用以下内容复制上述内容：`as_retriever` `search_type` `search_kwargs`

In [18]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["猫", "鲨鱼"])

[[Document(page_content='猫是独立的宠物，经常享受自己的空间.', metadata={'source': '哺乳动物宠物-doc'})],
 [Document(page_content='金鱼是初学者的热门宠物，需要相对简单的照顾.', metadata={'source': '鱼类动物-doc'})]]

VectorStoreRetriever支持 （default）、（最大边际相关性，如上所述）和 我们可以使用后者通过相似性分数对检索器输出的文档进行阈值。

"similarity" "mmr" "similarity_score_threshold"



检索器可以很容易地合并到更复杂的应用程序中，例如检索增强生成 （RAG） 应用程序，它将给定的问题与检索到的上下文组合成 LLM 的提示。

下面我们展示一个最小的例子。

In [19]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="qwen-plus", base_url="https://dashscope.aliyuncs.com/compatible-mode/v1") # 还是用阿里好了

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [21]:
response = rag_chain.invoke("告诉我关于猫的信息")

print(response.content)

猫是独立的宠物，它们通常喜欢有自己的空间。


检索策略可以是丰富而复杂的。例如：

1. 我们可以从查询中推断出硬规则和过滤器（例如，“使用 2020 年之后发布的文档”）;

2. 我们可以以某种方式（例如，通过某些文档分类法）返回链接到检索到的上下文的文档;

3. 我们可以为每个上下文单元生成多个嵌入;

4. 我们可以汇总来自多个检索器的结果;

我们可以为文档分配权重，例如，将最近的文档权重更高。